In [4]:
# ==========================================
# Cell 1: Load requirements.json (with stakeholders)
# ==========================================
import json
import pathlib
import pandas as pd

def find_repo_root(start=pathlib.Path.cwd()):
    for cand in [start] + list(start.parents):
        if (cand / 'build.gradle').exists() or (cand / '.git').exists():
            return cand
    return pathlib.Path.cwd()

repo_root = find_repo_root()
req_path = repo_root / 'build' / 'results' / 'mission-control' / 'requirements.json'
if not req_path.exists():
    raise FileNotFoundError(f"Requirements file not found at {req_path} (repo_root={repo_root})")

with req_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
rows = []

def short_from_uri(u):
    if not u:
        return ''
    if '#' in u:
        return u.split('#')[-1]
    return u.rstrip('/').split('/')[-1]

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    desc = b.get('description', {}).get('value')
    expr = b.get('expression', {}).get('value')
    stakeholder_uri = b.get('stakeholderURI', {}).get('value')

    name = short_from_uri(req_uri)
    stakeholder = short_from_uri(stakeholder_uri)

    rows.append({
        'Name': name,
        'Description': desc,
        'Expression': expr,
        'Stakeholder': stakeholder
    })

df = pd.DataFrame(rows)
df

,Name,Description,Expression,Stakeholder
0,R6,Interoperability with Fire Cloud,The system shall ingest and interpret data str...,SystemEngineer
1,R1,Real-Time Map,The system shall display real-time fire and dr...,Operator
2,R2,Critical Alerts,The system shall show real-time critical alert...,SafetyOfficer
3,R7,Decision Approval Mechanism,The system shall allow human approval for AI F...,CommandCenter
4,R3,AI Warden Interface,"The system shall enable direct, smooth communi...",Operator
5,R5,Historical Views,The system shall support views of historical f...,Operator
6,R4,Adaptive User Experience,The system shall remain functional under field...,FireFighter


In [9]:
# ==========================================
# Cell 2: Build mission-capability-requirement graph
# ==========================================
import networkx as nx
import plotly.graph_objects as go

cap_path = repo_root / 'build' / 'results' / 'mission-control' / 'missions_capabilities.json'
if not cap_path.exists():
    raise FileNotFoundError(f"Capabilities file not found at {cap_path} (repo_root={repo_root})")

with cap_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])

nodes = {}
edges = set()

for b in bindings:
    req_uri = b.get('requirementURI', {}).get('value')
    req_desc = b.get('requirementDescription', {}).get('value')
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    sub_uri = b.get('subCapabilityURI', {}).get('value')
    sub_desc = b.get('subCapabilityDescription', {}).get('value')

    req_name = short_from_uri(req_uri)
    cap_name = short_from_uri(cap_uri)
    sub_name = short_from_uri(sub_uri) if sub_uri else None

    if req_name:
        if req_name not in nodes:
            nodes[req_name] = req_desc
    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if sub_name:
        if sub_name not in nodes:
            nodes[sub_name] = sub_desc

    # requirement → capability
    if req_name and cap_name:
        edges.add((req_name, cap_name))
    # capability → subcapability
    if cap_name and sub_name:
        edges.add((cap_name, sub_name))

G = nx.Graph()
for n, d in nodes.items():
    G.add_node(n, description=d)
for a, b in edges:
    G.add_edge(a, b)

pos = nx.spring_layout(G, seed=42)

# --- Build Plotly traces ---
edge_x, edge_y = [], []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code by prefix: R=Requirements (coral), C=Capabilities (lightblue), M=Missions (lightgreen)
    if n.startswith('R'):
        node_colors.append('LightCoral')
    elif n.startswith('C'):
        node_colors.append('LightSkyBlue')
    elif n.startswith('M'):
        node_colors.append('LightGreen')
    else:
        node_colors.append('LightGray')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color=node_colors, line=dict(width=1, color='DarkSlateGrey'))
)

fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Requirement–Capability–SubCapability Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig.show()


In [10]:
# ==========================================
# Cell 3: Capability → Entity Graph
# ==========================================
ent_path = repo_root / 'build' / 'results' / 'mission-control' / 'entities.json'
if not ent_path.exists():
    raise FileNotFoundError(f"Entities file not found at {ent_path} (repo_root={repo_root})")

with ent_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}
edges = set()

for b in bindings:
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    ent_uri = b.get('entityURI', {}).get('value')
    ent_desc = b.get('entityDescription', {}).get('value')

    cap_name = short_from_uri(cap_uri)
    ent_name = short_from_uri(ent_uri)

    if cap_name:
        if cap_name not in nodes:
            nodes[cap_name] = cap_desc
    if ent_name:
        if ent_name not in nodes:
            nodes[ent_name] = ent_desc
    if cap_name and ent_name:
        edges.add((cap_name, ent_name))

G_entities = nx.Graph()
for n, d in nodes.items():
    G_entities.add_node(n, description=d)
for a, b in edges:
    G_entities.add_edge(a, b)

pos = nx.spring_layout(G_entities, seed=42)

# --- Build edge trace ---
edge_x, edge_y = [], []
for u, v in G_entities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y, mode='lines',
    line=dict(width=1, color='#888'),
    hoverinfo='none'
)

# --- Build node trace ---
node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G_entities.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_entities.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code: capabilities (C prefix) = lightblue, entities (others) = lightgreen
    if n.startswith('C') and n[1:].isdigit():  # e.g., C1, C2, C3...
        node_colors.append('LightSkyBlue')
    else:  # entity names like SafetyOfficer, FireDrone, etc.
        node_colors.append('LightGreen')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=18, color=node_colors, line=dict(width=1, color='DarkSlateGrey'))
)

fig_entities = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Capability–Entity Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_entities.show()


In [11]:
# ==========================================
# Cell 4: Activities → Data Flow Graph
# ==========================================
act_path = repo_root / 'build' / 'results' / 'mission-control' / 'activities.json'
if not act_path.exists():
    raise FileNotFoundError(f"Activities file not found at {act_path} (repo_root={repo_root})")

with act_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}  # activity_name -> description
edges = {}  # (a1, a2) -> (data_uri, datadesc)

for b in bindings:
    a1_uri = b.get('a1', {}).get('value')
    a1_desc = b.get('desc1', {}).get('value')
    a2_uri = b.get('a2', {}).get('value')
    a2_desc = b.get('desc2', {}).get('value')
    data_uri = b.get('d', {}).get('value')
    datadesc = b.get('datadesc', {}).get('value')

    a1_name = short_from_uri(a1_uri)
    a2_name = short_from_uri(a2_uri)
    data_name = short_from_uri(data_uri)

    if a1_name:
        if a1_name not in nodes:
            nodes[a1_name] = a1_desc
    if a2_name:
        if a2_name not in nodes:
            nodes[a2_name] = a2_desc
    
    # Store edge with data URI and description
    if a1_name and a2_name:
        edges[(a1_name, a2_name)] = (data_name, datadesc)

G_activities = nx.DiGraph()  # Directed graph for activities
for n, d in nodes.items():
    G_activities.add_node(n, description=d)
for (a1, a2), (data_name, dd) in edges.items():
    G_activities.add_edge(a1, a2, data_name=data_name, datadesc=dd)

pos = nx.spring_layout(G_activities, seed=42)

# --- Build edge traces (lines only, no hover) ---
edge_x, edge_y = [], []
for u, v in G_activities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=2, color='#888'),
    hoverinfo='none',
    showlegend=False
)

# --- Build edge label trace (at midpoint of each edge) ---
edge_label_x, edge_label_y, edge_label_text, edge_label_hover = [], [], [], []
for u, v in G_activities.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    # Midpoint of edge
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    
    data_name = G_activities.edges[u, v].get('data_name', '')
    datadesc = G_activities.edges[u, v].get('datadesc', '')
    
    edge_label_x.append(mid_x)
    edge_label_y.append(mid_y)
    edge_label_text.append(data_name)
    edge_label_hover.append(f"{data_name}<br>{datadesc}")

edge_label_trace = go.Scatter(
    x=edge_label_x, y=edge_label_y,
    mode='markers+text',
    text=edge_label_text,
    textposition='middle center',
    textfont=dict(size=10, color='#555'),
    hovertext=edge_label_hover,
    hoverinfo='text',
    marker=dict(size=8, color='#FFD700', opacity=0.7, line=dict(width=0.5, color='#888')),
    showlegend=False
)

# --- Build node trace ---
node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G_activities.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_activities.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code activities (A prefix) with a distinct color
    if n.startswith('A') and n[1:].isdigit():
        node_colors.append('LightSalmon')
    else:
        node_colors.append('LightGray')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=20, color=node_colors, line=dict(width=1, color='DarkSlateGrey')),
    showlegend=False
)

fig_activities = go.Figure(
    data=[edge_trace, edge_label_trace, node_trace],
    layout=go.Layout(
        title='Activity–Data Flow Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_activities.show()


In [20]:
# ==========================================
# Cell 5: State Machine Graph
# ==========================================
sm_path = repo_root / 'build' / 'results' / 'mission-control' / 'statemachine.json'
if not sm_path.exists():
    raise FileNotFoundError(f"State machine file not found at {sm_path} (repo_root={repo_root})")

with sm_path.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}  # state_name -> description
edges = {}  # (s1, s2) -> (trigger_name, triggerdesc)

for b in bindings:
    s1_uri = b.get('s1', {}).get('value')
    s1_desc = b.get('s1desc', {}).get('value')
    s2_uri = b.get('s2', {}).get('value')
    s2_desc = b.get('s2desc', {}).get('value')
    trigger_uri = b.get('trigger', {}).get('value')
    trigger_desc = b.get('triggerdesc', {}).get('value')

    s1_name = short_from_uri(s1_uri)
    s2_name = short_from_uri(s2_uri)
    trigger_name = short_from_uri(trigger_uri) if trigger_uri else None

    if s1_name:
        if s1_name not in nodes:
            nodes[s1_name] = s1_desc
    if s2_name:
        if s2_name not in nodes:
            nodes[s2_name] = s2_desc
    
    # Store edge with trigger URI and description
    if s1_name and s2_name:
        edges[(s1_name, s2_name)] = (trigger_name, trigger_desc)

G_statemachine = nx.DiGraph()  # Directed graph for state transitions
for n, d in nodes.items():
    G_statemachine.add_node(n, description=d)
for (s1, s2), (trigger_name, td) in edges.items():
    G_statemachine.add_edge(s1, s2, trigger_name=trigger_name, triggerdesc=td)

pos = nx.spring_layout(G_statemachine, seed=42, k=0.5)

# --- Build edge traces (lines only, no hover) ---
edge_x, edge_y = [], []
for u, v in G_statemachine.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=2, color='#888'),
    hoverinfo='none',
    showlegend=False
)

# --- Build edge label trace (at midpoint of each edge) ---
edge_label_x, edge_label_y, edge_label_text, edge_label_hover = [], [], [], []
for u, v in G_statemachine.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    # Midpoint of edge
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    
    trigger_name = G_statemachine.edges[u, v].get('trigger_name', '')
    triggerdesc = G_statemachine.edges[u, v].get('triggerdesc', '')
    
    if trigger_name:  # Only add label if trigger exists
        edge_label_x.append(mid_x)
        edge_label_y.append(mid_y)
        edge_label_text.append(trigger_name)
        edge_label_hover.append(f"{trigger_name}<br>{triggerdesc}")

edge_label_trace = go.Scatter(
    x=edge_label_x, y=edge_label_y,
    mode='markers+text',
    text=edge_label_text,
    textposition='middle center',
    textfont=dict(size=9, color='#555'),
    hovertext=edge_label_hover,
    hoverinfo='text',
    marker=dict(size=8, color='#90EE90', opacity=0.7, line=dict(width=0.5, color='#888')),
    showlegend=False
)

# --- Build node trace ---
node_x, node_y, hover_text, labels, node_colors = [], [], [], [], []
for n in G_statemachine.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G_statemachine.nodes[n].get('description') or ''
    hover_text.append(f"{n}<br>{desc}")
    labels.append(n)
    
    # Color-code states: Dashboard states vs Warden states
    if 'Dashboard' in n or n in ['Monitoring', 'Idle', 'AlertHandling', 'DecisionPending', 'HistoricalReview']:
        node_colors.append('LightBlue')
    elif 'Warden' in n or n in ['Observing', 'Advising', 'AwaitingApproval', 'ExecutingAction']:
        node_colors.append('LightCoral')
    else:
        node_colors.append('LightGray')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    textfont=dict(size=9),
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=22, color=node_colors, line=dict(width=1, color='DarkSlateGrey')),
    showlegend=False
)

fig_statemachine = go.Figure(
    data=[edge_trace, edge_label_trace, node_trace],
    layout=go.Layout(
        title='State Machine Transition Graph',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40)
    )
)

fig_statemachine.show()


In [14]:
# ==========================================
# Cell 6: Export all visuals into one HTML dashboard
# ==========================================
from pathlib import Path

table_html = df.to_html(classes='table table-striped table-bordered', index=False)
fig_missions_html = fig.to_html(full_html=False, include_plotlyjs=False)
fig_entities_html = fig_entities.to_html(full_html=False, include_plotlyjs=False)
fig_activities_html = fig_activities.to_html(full_html=False, include_plotlyjs=False)
fig_statemachine_html = fig_statemachine.to_html(full_html=False, include_plotlyjs=False)

html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Mission Control Dashboard</title>
    <link rel="stylesheet"
          href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <link rel="stylesheet"
          href="https://cdn.datatables.net/1.13.6/css/dataTables.bootstrap5.min.css">
    <script src="https://cdn.jsdelivr.net/npm/jquery@3.7.1/dist/jquery.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script src="https://cdn.datatables.net/1.13.6/js/dataTables.bootstrap5.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
      body {{ margin: 2rem; }}
      h1 {{ margin-bottom: 1.5rem; }}
    </style>
</head>
<body>
    <h1>Requirements Table</h1>
    <div class="table-responsive">
      {table_html}
    </div>

    <h1 style="margin-top:3rem;">Requirement–Capability–SubCapability Graph</h1>
    {fig_missions_html}

    <h1 style="margin-top:3rem;">Capability–Entity Graph</h1>
    {fig_entities_html}

    <h1 style="margin-top:3rem;">Activity–Data Flow Graph</h1>
    {fig_activities_html}

    <h1 style="margin-top:3rem;">State Machine Transition Graph</h1>
    {fig_statemachine_html}

    <script>
    $(document).ready(function() {{
        $('table').DataTable({{
            paging: true,
            searching: true,
            ordering: true,
            pageLength: 10
        }});
    }});
    </script>
</body>
</html>
"""

out_path = Path("mission_dashboard.html")
out_path.write_text(html, encoding="utf-8")
print(f"✅ Dashboard saved to {out_path.resolve()}")


✅ Dashboard saved to /Users/vivek/Notes/CS_188/mission-control/notebooks/mission_dashboard.html
